In [1]:
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller
import re
import sqlite3

conn = sqlite3.connect('data.db')
c = conn.cursor()

# Create technologies table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='techs' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE techs(
            id varchar NOT NULL PRIMARY KEY,
            regex varchar NOT NULL,
            type varchar NOT NULL DEFAULT 'language'
        )
    ''')
    c.execute('''CREATE UNIQUE INDEX techs_unique_name ON techs(id)''')
    c.execute('''CREATE UNIQUE INDEX techs_unique_regex ON techs(regex)''')
else:
    print("Table techs exists")

# Create positions table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='positions' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE positions(
            id integer NOT NULL PRIMARY KEY,
            name varchar NOT NULL,
            salary real NULL,
            company varchar,
            source varchar NOT NULL
        )
    ''')
else:
    print("Table positions exists")

# Create linker
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='positions_techs' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE positions_techs(
            tech_id varchar,
            position_id integer,
            PRIMARY KEY (tech_id, position_id),
            FOREIGN KEY (tech_id) REFERENCES techs(id) ON DELETE CASCADE,
            FOREIGN KEY (position_id) REFERENCES positions(id) ON DELETE CASCADE
        )
    ''')
else:
    print("Table positions_techs exists")

# Add Programming Languages
c.execute('''INSERT OR IGNORE INTO techs (id, regex) VALUES
('Javascript', 'javascript'),
('Java', '[ ,]java[ \.,]'),
('C', '[ ,]c[ \.,]'),
('C++', 'c\+\+'),
('Kotlin', 'kotlin'),
('Swift', 'swift'),
('Ruby', 'ruby'),
('Python', 'python3?'),
('SQL', 'sql'),
('R', '[ ,]r[ \.,]'),
('Perl', 'perl'),
('C#', 'c#'),
('Lua', 'lua'),
('Rust', 'rust'),
('PHP', 'php'),
('HTML', 'html'),
('CSS', 'css'),
('XML', 'xml')
''')

conn.commit()

def load_techs():
    c.row_factory = sqlite3.Row
    c.execute(''' SELECT * FROM techs ''')
    techs = {}
    for record in c.fetchall():
        techs[record['id']] = record['regex']
    return techs

technologies = load_techs()

In [ ]:
# Code for data collection
from bs4 import BeautifulSoup as bs
import requests as rq

langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}

page = rq.get("https://www.linkedin.com/jobs/search?keywords=Software%2Bengineer&location=United%20States&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")
soup = bs(page.content, 'html.parser')
items = soup.find_all(class_="base-card__full-link")
print(langs.values())
salary_match = "\$?\d{0,3},?\d{1,3},\d{3,3}\.?\d{0,2}"
item = items[0]
for item in items:
    url = item.get('href')
    print(url + "\n")
    posting_page = rq.get(url)
    s = bs(posting_page.content, 'html.parser')
    text = s.get_text()
    sal_matches = list(map(lambda x: float(re.sub("[^0-9\.]", "", x)), re.findall(salary_match, text)))
    for k, v in langs.items(): 
        if re.search(k, text, flags=re.IGNORECASE) != None:
            langs[k].extend(sal_matches)
    
print(langs)

dict_values([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])
https://www.linkedin.com/jobs/view/software-engineer-entry-level-at-nextcapital-3056825922?refId=wsQZgV%2Buuq7vngTcWVBheg%3D%3D&trackingId=oeJFMgmpfjfJhO0jDXP6fw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/software-engineer-at-underdog-media-3051588408?refId=wsQZgV%2Buuq7vngTcWVBheg%3D%3D&trackingId=vsnsiH9Z%2FHjF63YRcRa7fQ%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/software-engineer-backend-at-rubrik-inc-3051552545?refId=wsQZgV%2Buuq7vngTcWVBheg%3D%3D&trackingId=Xm%2BaQ7Bs00Kf7IR7ZIXj2g%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/software-engineer-at-cocoon-3026865276?refId=wsQZgV%2Buuq7vngTcWVBheg%3D%3D&trackingId=%2B8pc%2BSqOT3v0rlvgA19I3A%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.l

In [ ]:
# Clean up old entries from running script
SOURCE = 'glassdoor'
c.execute(f"DELETE FROM positions WHERE source='{SOURCE}'")

scraped_jobs = []
job_set = set()

# Initialize Driver
url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)
keyboard = Controller()

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    keyboard.press(Key.esc)
    keyboard.release(Key.esc)
    
    num_pages_to_search = 1
    num_pages_searched = 0
    num_jobs_searched = 0
    
    # extract the job data in "num_pages_to_search" pages
    while num_pages_searched < num_pages_to_search:
        # Each job
        jobs_on_page = driver.find_elements_by_class_name('react-job-listing')
        
        for job_card in jobs_on_page:
            # sleep to load the info
            sleep(1)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)

            job_description = driver.find_element_by_class_name('jobDescriptionContent').text.lower() # THIS IS JOB DESCRIPTION
            techs = extract_technologies(job_description) # TODO: Extract Programming Languages from Job Description
            salary = process_salary(driver)
            company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
            job_position = driver.find_element_by_class_name('css-1j389vi').text
            save_data({'company': company, 'salary': salary, 'position': job_position, 'technologies': techs}, source=SOURCE) # Format information

            job_card.click()

        sleep(1)
        keyboard.press(Key.esc)
        keyboard.release(Key.esc)
        keyboard.press(Key.esc)
        keyboard.release(Key.esc)

        job_description = driver.find_element_by_class_name('jobDescriptionContent').text.lower() # THIS IS JOB DESCRIPTION
        techs = extract_technologies(job_description) # TODO: Extract Programming Languages from Job Description
        salary = process_salary(driver)
        company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
        job_position = driver.find_element_by_class_name('css-1j389vi').text
        save_data({'company': company, 'salary': salary, 'position': job_position, 'technologies': techs}, source=SOURCE) # Format information

        num_pages_searched += 1
        if num_pages_searched == num_pages_to_search:
            print("Finished Searching. Check the salaries.csv file for output")
        
        # go to next page
        driver.find_element_by_class_name('nextButton').click()
        sleep(5)

# Helper function to extract programming languages from the job description text
def extract_technologies(job_description):
    temp = []
    for tech, reg in technologies.items():
        if re.search(reg, job_description, flags=re.IGNORECASE) is not None:
            temp.append(tech)
    return temp
    
# Used to get the name of Job company without star ratings
def process_company(company):
    try:
        if driver.find_element_by_class_name('css-1m5m32b') is not None:
            return company[0:-4]
    except NoSuchElementException:
        return company

# $70,000 /yr (est.) => 70000
# $35 /hr (est.) => 35*52*40
def process_salary(driver):
    try:
        salary = driver.find_element_by_class_name('css-y2jiyn').text
        new_salary = salary.split(' ', 1)
        if new_salary[1] == '/yr (est.)':
            return new_salary[0][1:]
        else:
            return str(float(new_salary[0][1:]) * 52 * 40)
    except NoSuchElementException:
        return None
        
# Format the data received from parsing a Job Card
def format_data(job_row):
    if job_row not in job_set:
        scraped_jobs.append(job_row)
        job_set.add(job_row)

def save_data(data, source=''):
    print("saving")
    position_query = "INSERT INTO positions (name, salary, company, source) VALUES (?, ?, ?, ?) RETURNING id"
    c.execute(position_query, (data['position'], data['salary'], data['company'], source))
    new_id = c.fetchone()[0]
    print(new_id)
    linkers = []
    tech_query = "INSERT INTO positions_techs (position_id, tech_id) VALUES (?, ?)"
    for tech in data['technologies']:
        linkers.append((new_id, tech))
    c.executemany(tech_query, linkers)
    conn.commit()
        
# Run my main program
main()
